In [1]:
# !pip install sastrawi

In [2]:
import re
import nltk
import os
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd '/content/drive/My Drive/Colab Notebooks/data_skripsi'

/content/drive/My Drive/Colab Notebooks/data_skripsi


In [29]:
!ls

 actual_word			   kata_dasar_db.txt
 hasil_idris			   PENGUJIAN_IDRISnew.ipynb
 hasil_inidris			   PENGUJIAN_INIDRISnew.ipynb
'Hasil perbandingan Idris.csv'	  'Perbandingan Stemming Idris.csv'
'Hasil perbandingan Inidris.csv'  'Perbandingan Stemming Inidris.csv'
 IDRIS.ipynb			   stopwords2.txt
 INIDRIS.ipynb			   stopwords.txt
 katadasarbahasaindonesia.txt	   test_data


In [30]:
stopword = open("stopwords2.txt", 'r').read() 
stopword = stopword.split("\n")
stopword[:5]

['ada', 'adalah', 'adanya', 'adapun', 'agak']

In [31]:
kata_dasar = open("kata_dasar_db.txt", 'r').read()
kata_dasar = kata_dasar.split("\n")

kamus = []
for i in kata_dasar:
    kata = "{}".format(i)
    kata = kata.replace("(Nomina)", "")
    kata = kata.replace("(Adjektiva)", "")
    kata = kata.replace("(Adverbia)", "")
    kata = kata.replace("(Numeralia)", "")
    kata = kata.replace("(Pronomina)", "")
    kata = kata.replace("(Verba)", "")
    kata = kata.replace("(Preposisi)", "")
    kata = kata.replace("(Interjeksi)", "")
    kata = kata.replace("(Konjungsi)", "")
    kata = kata.replace("(Lain-lain)", "")
    kata = kata.replace("\n", "")
    kata = kata.rstrip()
    kamus.append(kata)
kamus[:5]

['a', 'ab', 'aba', 'aba-aba', 'abad']

In [32]:
# cek kata dasar di kamus
def cekKamus_idris(kata):
    if kata in kamus:
        return kata
    else:
        status = 0
    return status

In [33]:
# fungsi untuk menghapus suffix seperti -ku, -mu, -kah, dsb
def del_inflection_suffixes_idris(kata):
    kataAsal = kata
    if(re.search(r"([kl]ah|pun)$", kata, re.IGNORECASE)):
        _kata = re.sub(r"([kl]ah|pun)$", '', kata, re.IGNORECASE)
        if(cekKamus_idris(_kata)):
            return _kata
        else:
            kata = _kata
    if(re.search(r'([km]u|nya)$', kata, re.IGNORECASE)):
        _kata = re.sub(r'([km]u|nya)$', '', kata, re.IGNORECASE)
        if(cekKamus_idris(_kata)):
            return _kata
    return kataAsal

In [34]:
# Cek Prefix Disallowed Sufixes (Kombinasi Awalan dan Akhiran yang tidak diizinkan)
def cek_prefix_disallowed_sufiks_idris(kata):
    kataAsal = kata
    # r"^(ber)[[:alpha:]](lah|an)$"
    if(re.search(r"^(ber)[^\W\d_](lah|an)$", kata, re.IGNORECASE)):
        _kata = re.sub(r"^(ber)", '', kata, re.IGNORECASE)
        return _kata
    # ^(men|di|pe|ter)[[:alpha:]](i)
    if (re.search(r"^(men|di|pe|ter)[^\W\d_](i)$", kata, re.IGNORECASE)):
        _kata = re.sub(r"^(men|di|pe|ter)", '', kata, re.IGNORECASE)
        return _kata
    return kataAsal

In [35]:
# Hapus Derivation Suffixes ("-i", "-an" atau "-kan")
def del_derivation_suffixes_idris(kata):
    if(re.search(r"(i|an)$", kata, re.IGNORECASE)):
        _kata = re.sub(r"(i|an)$", '', kata, re.IGNORECASE)
        if(cekKamus_idris(_kata)):
            return _kata
        elif(re.search(r"(kan)$", kata)):
            _kata = re.sub(r"(kan)$", '', kata, re.IGNORECASE)
            if(cekKamus_idris(_kata)):
                return _kata
            else:
                _kata = del_derivation_prefix_idris(_kata)
                return _kata

In [36]:
# Hapus Derivation Prefix ("di-", "ke-", "se-", "te-", "be-", "me-", atau "pe-")
def del_derivation_prefix_idris(kata):
    kataAsal = kata
    if(re.search(r"^(diper)", kata, re.IGNORECASE)):
        _kata = re.sub(r"^(diper)", '', kata, re.IGNORECASE)
        return _kata

    if(re.search(r"^(di|[ks]e)", kata, re.IGNORECASE)):
        _kata = re.sub(r"^(di|[ks]e)", '', kata, re.IGNORECASE)
        return _kata

    if(re.search(r"^(ke[bt]er)", kata, re.IGNORECASE)):
        _kata = re.sub(r"^(ke[bt]er)", '', kata, re.IGNORECASE)
        return _kata

    if(re.search(r"^([bt]e)", kata, re.IGNORECASE)):
        _kata = re.sub(r"^([bt]e[lr])", '', kata, re.IGNORECASE)
        if(cekKamus_idris(_kata)):
            return _kata
        _kata = re.sub(r"^([bt]e)", '', kata, re.IGNORECASE)
        return _kata

    if(re.search(r"^([mp]e)", kata, re.IGNORECASE)):
        _kata = re.sub(r"^([mp]e)", '', kata, re.IGNORECASE)
        if(cekKamus_idris(_kata)):
            return _kata
        else:
            if(re.search(r"^(memper)", kata, re.IGNORECASE)):
                _kata = re.sub(r"^(memper)", '', kata, re.IGNORECASE)
                if(cekKamus_idris(_kata)):
                    return _kata

            if(re.search(r"^([mp]eng)", kata, re.IGNORECASE)):
                _kata = re.sub(r"^([mp]eng)", '', kata, re.IGNORECASE)
                if(cekKamus_idris(_kata)):
                    return _kata
                else:
                    _kata = re.sub(r"([mp]eng)", 'k', kata, re.IGNORECASE)

            if(re.search(r"^([mp]eny)", kata, re.IGNORECASE)):
                _kata = re.sub(r"^([mp]eny)", 's', kata, re.IGNORECASE)

            if(re.search(r"^([mp]e[lr])", kata, re.IGNORECASE)):
                _kata = re.sub(r"^([mp]e[lr])", '', kata, re.IGNORECASE)

            if(re.search(r"^([mp]en)", kata, re.IGNORECASE)):
                _kata = re.sub(r"^([mp]en)", 't', kata, re.IGNORECASE)
                if(cekKamus_idris(_kata)):
                    return _kata
                else:
                    _kata = re.sub(r"^([mp]en)", '', kata, re.IGNORECASE)

            if(re.search(r"^([mp]em)", kata, re.IGNORECASE)):
                _kata = re.sub(r"^([mp]em)", '', kata, re.IGNORECASE)
                if(cekKamus_idris(_kata)):
                    return _kata
                else:
                    _kata = re.sub(r"^([mp]em)", 'p', kata, re.IGNORECASE)

            return _kata
    return kataAsal

In [37]:
def step2_idris(kata):
    kataAsal = kata
    kata = cek_prefix_disallowed_sufiks_idris(kata)
    if(cekKamus_idris(kata)):
        return kata
    else:
        kata = del_derivation_prefix_idris(kata)
        if(cekKamus_idris(kata)):
            return kata
        else:
            kata = del_inflection_suffixes_idris(kata)
            if(cekKamus_idris(kata)):
                return kata
            else:
                kata = del_derivation_suffixes_idris(kata)
                # print("4"+str(kata))
                if(cekKamus_idris(kata)):
                    return kata
                else:
                    return kataAsal+" "+"(Kata Salah)"

In [38]:
def stemming_idris(kata):
    kataAsal = kata
    # cekKamus = cekKamus_idris(kata)
    if (cekKamus_idris(kata) != 0):
        return kata
    else:
        kata = step2_idris(kata)
        return kata

In [39]:
# Teks = "Aku Tak Sengaja Memusnahkan Guru Pra-Aljabarku. Aku tak pernah ingin jadi anak blasteran. Kalau kau membaca cerita ini karena menduga kau anak blasteran juga, kunasihati nih: tutup buku ini sekarang juga. Pokoknya percayai saja apa pun kebohongan yang diceritakan ayah-ibumu tentang kelahiranmu, dan cobalah menjalani hidup normal. Menjadi blasteran itu berbahaya. Menyeramkan. Sering bikin orang terbunuh secara menyakitkan dan mengerikan. Kalau kau anak normal, yang membaca ini karena menganggap ini fiksi, bagus. Silakan baca terus. Aku iri denganmu, bisa percaya bahwa semua peristiwa ini tak pernah terjadi. Tapi, kalau kau mengenali dirimu di halaman-halaman ini—kalau kau merasa sesuatu menggeliat dalam dirimu—cepat hentikan bacaanmu. Mungkin saja kau sama seperti kami. Dan begitu kau menyadari itu, cuma soal waktu saja sampai mereka juga merasakannya, dan mereka akan memburumu. Jangan bilang aku tak pernah memperingatkanmu. Namaku Percy Jackson. Umurku dua belas tahun. Hingga beberapa bulan yang lalu, aku siswa asrama di Akademi Yancy, sekolah swasta untuk anak bermasalah di New York Utara. Apa aku anak bermasalah? Yah. Bisa dibilang begitu. Kalau mau bukti, aku bisa mulai bercerita dari titik mana pun dalam hidup pendekku yang mengenaskan ini, tetapi keadaan mulai benar-benar memburuk pada bulan Mei lalu, sewaktu murid kelas enam kami berkaryawisata ke Manhattan—dua puluh delapan anak sakit jiwa dan dua orang guru naik bus sekolah kuning, menuju Museum Seni Metropolitan untuk melihat barang-barang Yunani dan Romawi Kuno. Iya, aku juga tahu—kedengarannya seperti penyiksaan. Sebagian besar karyawisata Danau merupakan sebuah tempat yang berbentuk lembah yang berada di daratan. Lembah tersebut dipenuhi oleh air. Keadaan airnya sangat jernih serta menyegarkan.Air yang berada di dalam danau tak pernah habis. Ketika musim penghujan air yang ada di dalam danau akan melimpah. Sementara ketika musim kemarau, air danau akan nampak surut.Kejernihan yang ada di air danau membuatnya kerap kali dimanfaatkan untuk dijadikan sebagai irigasi pertanian. Dan juga dimanfaatkan sebagai sumber mata air di seluruh kawasan sekitarnya. Oleh sebab itulah, danau ini menjadi sumber untuk kehidupan seluruh mahluk.Di tepian danau tumbuh beberapa tumbuhan air contohnya bunga teratai. Dengan kehadiran dari bunga teratai itu membuat danau akan lebih indah. Tepat di bawah teratai, pada umumnya akan ditemukan berbagai spesies ikan. Mulai dari ikan koi, ikan mas dan yang lainnya.Namun kadang kali tepat di atas danau nampak beberapa spesies burung seperti angsa. Para angsa juga seakan tengah menikmati suasana alam danau yang amat menyejukan dengan berenang berkeliling danau. Kadang para angsa tersebut juga memburu ikan disana. Di dalam rumah ada seekor kucing jenis Persia yang bernama Milo. Milo merupakan satu – satunya kucing kesayanganku. Milo memiliki sifat yang aktif dan cukup manja. Ia suka sekali dibelai dengan lembut sembari tidur di pangkuanku. Salah satu kebiasaan Milo yakni selalu menyambutku ketika pulang ke rumah. Ia bahkan selalu stay di depan pintu ketika aku membuka pintu. Lalu ia pun akan segera mengikutiku menuju kamar serta meletakkan tas di meja. Sebab setiap hari aku harus belajar, aku pun langsung berbaring di tempat tidur selepas pulang. Milo sekali menemaniku ketika tiduran. Aku pun tak pernah lupa untuk mengelus – elus kepalanya. Kadang kali kita berdua tak sengaja tertidur sampai larut malam. Tak hanya suka berada di dekatku, Milo juga memiliki hobi makan. Porsi makanan Milo bahkan jauh lebih besar dibandingkan dengan porsi makananku. Ia sangat gemar dengan keju cheddar. Namun Milo juga suka sekali dengan susu. Namun aku lebih sering memberinya berupa makanan kemasan. Hal tersebut aku lakukan guna menekan biaya pengeluaran. Karena harga keju itu lebih mahal dibandingkan dengan harga makanan kucing kemasan. Sehingga hanya sesekali aku akan memberinya makanan berupa keju. Ketika Milo memakan keju, ia akan memakannya dengan sangat lahap hingga menghabiskan hampir 500 gr keju untuk sekali makan. Sementara untuk susu, memang aku berikan dengan rutin. Milo memiliki bulu yang cukup panjang dan lebat. Warna bulu Milo adalah putih. Tubuh Milo juga sangat gemuk dan juga berisi. Hidung Milo sangat pesek, bahkan hampir tidak terlihat serta tersamarkan dengan wajahnya yang chubby. Milo merupakan hadiah pemberian dari nenek pada ulang tahunku yang ke 10 tahun. Pada mulanya aku memang tak menyukai hadiah tersebut, namun selepas mengenal sifat Milo aku menjadi sangat sayang kepadanya. Karakter Milo juga tidak terlalu liar. Ia justru termasuk ke dalam jenis kucing pendiam serta tidak banyak tingkah. Itulah mengapa aku sangat menyukai Milo. Rumahku mempunyai ukuran yang tak terlalu besar namun juga tak terlalu kecil. Rumah satu ini merupakan milik keluargaku yang sudah dibeli jauh sebelum ayah menikah bersama ibu. Rumah sederhana ini berada di depan mushola di blok pertama dari arah depan gerbang kompleks perumahan Citra Kirana. Rumah kami mempunyai luas 50 meter persegi. Terdiri atas 3 kamar serta 2 kamar mandi. Rumah ini juga tidak mempunyai memiliki lantai atas, hanya terdapat halaman yang cukup guna dijadikan sebagai kebun sederhana Pada saat pertama kali kita masuk ke bagian depan, maka akan disambut dengan ruang tamu yang dilanjutkan dengan kamar pribadi lalu terdapat satu dapur serta satu ruang santai. Untuk bagian ruang keluarga sendiri berada di jalan ke arah kamar yang ada TV serta tempat duduk bersantai. Di ruang tamu juga hanya berisikan alas karpet serta tempat duduk dan juga rak buku yang di tempel di dinding. Seluruh barang yang berada di rumah selalu tertata dengan rapi. Rumah kami diwarnai dengan cat berwarna hijau muda, sehingga membuatnya nampak sejuk dipandang mata. Halamannya terletak di depan penuh dengan tanaman yang telah ditanam memakai sistem hidroponik. Beragam jenis tanaman yang di tanam pada kebun mini ini diantaranya seperti sayur kangkung, cabe, sawi dan bawang merah serta putih. Seluruh jenis tanaman tersebut sudah berbuah banyak. Aku sangat nyaman sekali dengan rumahku. Jam tangan merupakan suatu aksesoris yang dipakai pada pergelangan tangan, sama halnya seperti memakai gelang tangan. Namun aksesoris jam tangan satu ini jauh lebih efektif serta fungsional. Tak hanya itu saja, jam tangan juga bisa dipakai sebagai alat pendukung penampilan. Pada jam tangan terdiri atas 2 bagian berbeda, yakni bagian kepala dan pengikat. Seluruh bagian pada jam tangan memiliki peran penting sendiri – sendiri. Meskipun sebetulnya bagian paling utama berupa bagian kepala jam itu sendiri. Di bagian kepala jam, secara garis besar terdiri atas 2 jenis tampilan. Tampilan pertama berupa tampilan klasis yang memiliki jarum analog. Sementara tampilan yang kedua berupa tampilan digital. Dimana tampilan digital ini telah dilengkapi dengan menggunakan layar LCD. Kadang juga ada yang memakai looks analog serta look jam digital. Baik jam analog maupun digital, kadang juga telah dilengkapi dengan kalender serta hari. Pada bagian samping, ada sebuah tombol guna mengatur waktu. Sementara dibagian pengikat memiliki design yang variatif. Ada yang memakai pita, karet, magnet, kulit, dan yang lainnya. Sebagai aksesoris yang fungsional, sekarang jam juga sudah menjadi salah satu benda yang inovatif. Bahkan sudah terdapat smartphone yang memiliki ukuran sebesar jam tangan. Cara memakainya pun sama dengan memakai jam tangan. Lalu pada jam tangan juga sudah dilengkapi dengan fitur waterproof. Sehingga tak mudah mati walaupun terkena air serta sangat aman untuk dipakai ketika hujan ataupun berenang. Oleh sebab itu, fitur waterproof dalam jam ini menjadi fitur favorit. Bahkan juga menjadi fitur utama yang harus ada pada setiap jam tangan. Aku ingin bercerita ujarku padanya hari itu Setelah lupa pada janjiku sendiri untuk tidak lagi menghubunginya aku dan ia kini terjebak dalam durasi panjang percakapan telepon jarak jauh  16170 kilometer dua puluh enam jam perjalanan udara Tidak ada pertanyaan tentang kabar tinggi matahari di kotanya tentang ramalan cuaca atau tentang harga terkini makanan instan hasil impor dari Tiongkok  terutama makanan kucing yang sering kami bicarakan Kami samasama tahu tidak ada satu unsur mekanis apapun yang bisa membuat kami tidak saling mengangkat telepon Barangkali kebiasaan adiktif ini membuat alam bawah sadar kami enggan lelap satu dering telepon dan cukup membuat kami terjaga belasan jam Kamu ingat apa yang kubilang soal ceritacerita yang kau tulis di rubik online itu Atau di blog pribadimu Entah kenapa tiap kali membaca ceritamu aku seperti terseret ke lubang hitam dan merasa hidup di dalam narasimu Kesialankulah aku baru saja membaca tulisan terbarumu dua jam lalu Tulisanmu sekuat itu memengaruhiku isi kepalaku gerak tanganku atau bahkan kehidupanku sendiri Semua tulisanku memang untukmu kok Sejak awal jawabnya Singkat dan menusuk sampai ulu hati Itu seakan memaksaku untuk terus tinggal di dalam sana saja Jadi semacam dunia tempat sembunyi Bahkan lucunya aku mengonversi itu jadi nyata meski aku tahu tidak semestinya demikian Dan harusnya dari awal kamu tanya kenapa aku cinta mati sama kamu Aku mendengar sayup suaranya mengumpat ke langitlangit kamar atau mungkin pada desau angin yang mengembus debudebu di tirai jendela Kalimat terakhirnya itu membuat udara New York dan Jakarta beku seketika pepat menyesakkan rongga dada Seharusnya aku cukup tahu diri agar kalimat seperti itu tidak perlu keluar dari mulutnya Kami bukan lagi sepasang kekasih jarak jauh Tema itu sudah selesai sejak dua bulan lalu ketika dengan mudahnya dia mengumpatku begitu kasar dengan nama binatang dan serta merta aku memutuskan kami harus berhenti Tapi entah apa arti berhenti bila salah satunya cinta mati dan yang lainnya berkepala besi Tidak bisa kupungkiri seperti termanipulasi sempurna aku mengonversi semua narasinya jadi nyata dari mulai menghisap rokok yang sama dengannya mulai menyukai anggur putih dan whiski  meski dahulu aku melarang keras ia untuk menegaknya menjadi makhluk malam yang terjaga di depan toko sudut gang sampai pagi menjadi robot yang hatinya bolong atau menjadi mesin pembalas pesan romansa bagi orangorang depresi yang kesepian di media anonim Aku menjadi rumah dua puluh empat jam yang bebas dikunjungi dan ditinggalkan siapapun  dengan jejakjejak sepatu yang kotor dan tetes air dari payung yang basah Aku terjebak di dalam sana bersama caranya menguasaiku Barangkali ini nasib sial bagi sesiapa yang pernah dicintai seorang penulis Semua peran itu pernah hidup di dalam tulisantulisannya Ada dunia di dalam sana dan aku adalah pemeran utamanya yang bisa bergantiganti wajah Aku seperti terbelah jadi banyak bahkan saat sekarang bicara dengannyapun rasanya aku sedang menjadi tokoh dalam cerita yang lain Tidak ada suaranya dari ujung sambungan telepon setelah itu meski aku tahu ia masih mendengar ceracauku dengan saksama Ia mendengarkanku fokus lebih fokus daripada prajurit di barisan depan batalyon perang mendengar perintah komandannya Memang seperti itulah adanya Ceritacerita yang ditulisnya nyatanya memberiku ruang untuk hidup Dia selalu menulis karenaku dan untukku Persis seperti citacitanya untuk menumbuhkan ekosistem dan pengalaman dengan harapan mampu mengubahku dari mesin rusak menjadi mesin baru Tapi kau manusia Kau bukan mesin Narasi yang kuciptakan untukmu semestinya tidak begini Untuk membuatmu hidup benar tapi tidak dalam dunia yang sembunyi dan purapura Aku mencintaimu dan itulah perasaan yang kupersembahkan untukmu seperti upeti untuk tuan puteri Tapi toh apa gunanya Kau bisa mudah menemukan lelaki mana saja yang dengan senang hati melayanimu Sambungan telepon ditutup Satu jam sembilan belas menit tiga puluh satu detik Sesuatu bergemuruh di dalam diriku merambat dari selasela tulang punggung menekan asam lambung memenuhi relung bergetar tanpa bisa kudefinisi apa dan mengapa Tapi aku tahu betul perasaan seperti ini hanya bekerja saat aku bicara dengannya atau mengingatnya atau berada di pelukannya menghirup aroma pewangi binatu yang bercampur dengan khas keringatnya Pelukan yang kurindukan rapatrapat Perkataannya bahwa tidak ada seorang lain di dunia ini yang mampu mencintaiku seperti ia mencintaiku kadang terasa seperti kutukan Mengenalnya bahkan membuatku percaya pada pepatah yang bilang kau bisa memilih dengan siapa kau akan menikah tapi tidak kepada siapa kau jatuh cinta Tapi sungguh aku tidak ingin jatuh cinta pada lelaki yang mengataiku anjing apapun alasannya Pada akhirnya kami adalah sepasang kekasih yang akan terus saling menatap dengan peluk meski berkalikali saling mematahkan Menyayangi dalam umpatanumpatan yang terdengar lebih indah dan ceria dari Prelude in C Major Dalam hidupku dirinya adalah kerumitan semesta percikpercik api amarah ketidaknyamanan yang paling kubenci sekaligus mempesona seperti kuncup bunga yang mekar di sore hari selepas hujan Gila membuatku jatuh cinta sekaligus mati dalam saat yang bersamaan Kemarin pagi pesawatnya mendarat dari New York sampai di Jakarta tepat pada pukul tujuh lewat dua puluh menit Dia tidak mengabari apaapa selain dua puluh delapan jam sebelumnya mengunggah gambar pada instagram story berisi salam perpisahan dengan kota yang sudah tiga tahun terakhir ditinggalinya dan dicintainya itu Kedatangannya ke Jakarta adalah berita paling menyakitkan dalam beberapa bulan terakhir Ada pesan keterpaksaan yang mengisi ruangruang kosong di udara bahwa dia menolak berada di satu kota yang sama denganku Ini terlalu dekat dan bisa saja kami berpapasan di trotoar jalan bertemu di bus kota atau tanpa sengaja mengunjungi pagelaran seni yang sama di panggung budaya paling terkenal kota ini Tempattempat favoritnya adalah tempattempat favoritku jua dan kami samasama ingin menghindari kecanggungan itu Dia datang ke kota ini demi memenuhi janjinya sendiri untuk menjagaku dalam jarak terdekat yang ia bisa Tapi sayangnya janji apapun tidak akan berjalan dengan semestinya kali ini Aku dan dia tidak menemukan alasan yang cukup untuk saling menjaga setelah kami berhenti menjadi sepasang kekasih Hari ini aku masih memikirkan skenario terbaik bila suatu hari tibatiba alam bercanda mempertemukan kami Hampir tiga bungkus rokok rasa madu habis kunyalakan di balkon kamar dan kepulkepul asapnya tidak juga memberiku inspirasi apapun selain kerumitan jiwa yang pernah mencintai lalu patah Aku menunggu ada labalaba atau hantu yang melintas lalu memberitahuku cara mengelabui canda semesta Sampai tibatiba pintu kamarku terketuk keras Dia ada di sana Sialan Kapan nasib buruk ini selesai Aku gila jatuh cinta pada perempuan di depanku ini dan sampai kapanpun aku nggak akan bisa membuatmu patah hati Setidaknya beri aku satu pelukan lagi Hey kau ingin apa Biar kuberikan segala yang aku punya dan tak punya untukmu"
# Teks = "MALAM membara membawaku untuk terus menanti kedatangan kereta warna biru Aku duduk memperhatikan saksama setiap petugas yang mengabarkan kedatangannya Kadangkadang muncul suara dari speaker Mohon perhatiannya akan datang dari jalur satu kereta ekspres Lagilagi aku mengeluh mungkinkah kereta yang harganya cuma tiga puluh lima ribu rupiah akan datang terlambat Batinku berkata sial Rasa kesalku akhirnya terobati saat kereta datang dari jalur dua Sudah satu jam lamanya aku menanti dan aku hanya berpendapat bahwa ular sangat kuat pun tak bisa mengalahkan baja yang dalam sedetik bisa melaju sangat kencangnya Sampai bebatuan melonjak tak pernah usai Cepat larinya bak pacuan kuda yang memenangkan lotre mingguan Butirbutir keringat jatuh dari mata dan daguku Sesekali keluh kesah terucapkan dengan nada kasar Panasnya malam setelah turun hujan tak ubahnya berjemur di padang sahara Wajahwajah dengan segala ekspresi telah terpampang rapi Aku berjalan bersinggungan badan antara perempuan dan lelaki Beberapa ibu dan anak kecil mencari gerbong yang agak sepi Rupanya ini tak terjadi hanya dalam realita pesta mudik dan Natal Ataukah ini dijadikan kesempatan untuk menghemat pengeluaran dalam bulanan Sungguh aku tak tahu Begitu kereta berhenti desakan koper dan jeritan manusia nyaring mengalahkan suara mesin Di sini aku mencoba berjalan pelan dengan membawa koper besar Seorang anak kecil menariknarik celana jeansku Kak Kakak orang kaya ya aku tak mengerti apa yang ditanyakannya GERBONG LIMA Aku hanya tersenyum tak terlalu mengerti apa yang ditanyakannya Pemenggalanpemenggalan kata yang terlalu rumit serumit semut mengatur pasukannya Masih ditambah ada bayi merengek tak tahu bahasanya sendiri Jelasnya aku masih sibuk mencari kursi kosong tak memedulikan beberapa pertanyaan dan omelan Dari jauh kulihat seorang perempuan berhenti sejenak hingga dia keluarkan handphone warna ungu Saatnya dia melintas di hadapanku terlihat menarik hati adakah magnet di kereta ini Tubuhnya beraroma wangi apricot Rambutnya seperti habis di rebounding lurus dan halus Baju kuningnya melekat sangat ketat Parasnya bulat penuh tata rias Mata gemintangnya dibingkai warna biru pucat Maskara telah melentikkan bulu matanya Kulihat bibir tipisnya berwarna shocking pink Dia sangat siap menggoda pria hingga salah satu pria merelakan tempat duduknya untuk perempuan tersebut Pandanganku tertuju kepada kursi C 01 Di sana ada perempuan yang tadi kulihat sangat menarik Dia memakai jaket keabuabuan Rapat matanya membuat orangorang mengira bahwa dia adalah orang China yang sedang menikmati libur musim hujan di kereta ekonomi tanpa AC dan TV Yang lebih parah dia membuka jaketnya lalu mengipasngipas Tak kubiarkan berahi ini memuncak dan kesalahan di masa lalu tak akan terulang lagi di sini batinku mencoba menahannya Lutut yang dia pamerkan kuanggap koran bekas Pandanganku telah berubah Kupalingkan mata kepada sosok di samping perempuan memesona itu Tepatnya di kursi C 02 Aku melihat lelaki yang tak kekar atau pun terlalu kurus Wajahnya terlihat sangat rapi dan dia memakai rompi ungu Semilir angin malam membuat rasa kantuk terhenti di pundak lelaki kekar yang duduk di kursi C 03 Tapi aku terusik suara nyaring pedagang asongan dia hanya menambah desakan dalam kereta Perangai anakanak yang kujumpai tadi masih terus saja menanyakan keadaanku bahkan soal cinta Kakak sudah nikah ya Ataukah sudah kawin Apakah katakata yang dimaksudkannya Nikah dan kawin Dua alur yang berbeda Duaduanya seperti belum kualami tapi salah satu dari pilihan itu terbayang seperti menghantui Tidak adik manis kedustaan yang menghujani otakku mulai terasa sakit Kembali teringat gambargambar yang tak begitu jelas Aku sulit mengatakannya bak menulis di atas lautan Dan akulah lautan itu Suasana gerah di malam hari walau pun hujan rintikrintik jatuh Ditambah kebisingan para penumpang aku tak berdaya melawan keadaan yang sangat menjengkelkan ini Saat berhenti di Stasiun Cirebon aku menguatkan langkah Kulawan desakan para penumpang menuju gerbong empat Mungkin di sana aku bisa mendapatkan tempat duduk dan segera pejamkan mata GERBONG EMPAT Kereta yang aku tumpangi memasuki Cirebon Beberapa stasiun di Jawa Barat kulalui Gelap malam semakin melarut Retakanretakan daun patah Aku mulai membaca doa tidur dan mimpi indah Tetapi beberapa kali handphoneku berbunyi Lalu lalang pesan seakan menjadi dampak akan berlebihan bergurau Aku mencoba membalas beberapa sms yang masuk Aku hanya menulis aku sibuk Kantukku terobati Di kursi D 01 kulihat ada seorang yang berkumis Aku tak tahu itukah lele yang didambakan setiap perempuan Ataukah perempuan mendambakan dada kekar dan anting kegagahan yang melekat di telinga kanannya Bahkan ada yang berbisik Aku lebih suka saat dia mengisap rokok Jiwa kasarnya seakan terlihat langsung Aku hanya mendengar bisikannya Tetapi ada yang menimpali bisikannya Kepulan asap itulah yang nanti membuat asmaku kambuh Apa dia mau membawakan dokter untukku aku agak setuju dengan komentar yang ini Lalu di belakangku muncul anak yang tadi kujumpai Kakak beruntung ya tak merokok Aku tak mengerti apakah yang dikatakannya Apakah dia menyindir ataukah sekadar kagum Lebihlebih aku heran kenapakah dia terus mengikutiku Tetapi biarlah toh dia hanya anak kecil yang kehilangan orang tua Aku memberinya permen dan cokelat Padahal rencanaku setelah sampai di Pekalongan itu akan kuberikan kepada anak tetanggaku yang usianya hampir sama dengannya Sedangkan di kursi D 02 kulihat seorang lakilaki yang hidupnya hanya memuji Tuhan Tentu aku tahu saat kulihat yang dia genggam bukan handphone atau MP4 melainkan sebuah tasbih melingkar warna putih Bibirnya masih saja dibasahi kalimat tayibah Sampai di sini pengihatanku tak nakal lagi karena tak ada perempuan semenarik tadi Dia adalah pemandangan terindah selama hidupku Aku berharap bisa mendapatkan kursi kosong untuk mengobati rasa kantukku yang mulai menjalar ke mata Tetapi akhirnya aku tertidur dengan gaya orang sedang buang air besar di wc umum Lanskap angkasa menunjukkan amukan amarahnya Beberapa petir menyambut kedatangan kereta di stasiun kecil Tegal Desau dedaunan jati apel dan bahkan mangga golek Wajahwajah boneka sama seperti yang pernah kututurkan sebelumnya Bau kotok ayam menyerbu hidung pesekku yang mungkin banyak komedo Saat itu para pedagang asongan mengeraskan teriaknya untuk mengalahkan suara mesin nasi hangat makan makan Ada juga suara pengemis uang barokah uang barokah Aku tak habis pikir di dunia secanggih ini masih ada pengemis yang seharian tak makan Beberapa koran lusuh tergeletak setelah para penumpang berhenti di Tegal Ini kesempatan bagiku untuk tidur di atas kursi Dan akhirnya malam mengizinkanku duduk setelah sekian lama berdiri Syukurlah ucapku lalu langsung duduk di kursi D 01 Wajah yang kulihat di kursi D 02 di gerbong empat berganti dengan wajah seorang ibu yang menggendong bayinya Umur berapa Bu tanyaku Tapi dia hanya mengangguk tanpa ada suara Hm bayi yang ibu gendong umur berapa Lagilagi tak ada jawaban Seorang asongan membuatku bernafsu untuk membeli mi seduh dengan aroma bawang Bombay Satu Mang Harganya tiga ribu kan Mang Satunya lima ribu jawabnya kasar Semenit sebelum aku merogoh saku tibatiba si ibu di sebelahku sudah duluan membayarnya Matur nuwun kataku Hujan masih terus mengguyur beberapa wilayah di Jawa Tengah Benakku bertanyatanya Akankah sampai di tujuan dengan selamat Di tengah kesyahduan yang menjemput tidur seorang polisi penarik karcis datang dengan pose keangkuhannya Di atas sakunya terlihat beberapa lambang jabatan Berkilau menyipitkan mata Karcisku sudah kuberikan dan sosoknya mulai menjauh Kini tinggal si ibu di sebelahku terus saja menenangkan bayinya yang rewel Dia membuatku terganggu untuk mendengarkan MP4 Berisik anak ibu tetapi teguranku tak membuatnya patah semangat Aku kembali memandangi wajah di sampingku Ada rasa aneh antara aku dan dia Sepertinya aku pernah mengenali bahkan akrab kepadanya namun aku teringat bahwa ada luka di otakku Maaf apakah kita pernah bertemu tanyaku Jawabannya masih hampa Maaf apakah kita pernah bertemu sebelumnya aku mengulangi pertanyaanku dengan ditambahi kata Namun dia membisu acuh tak acuh Hanya bayinya yang dia pikirkan Bayi itu terus merontaronta seakan ingin bertemu dengan bapaknya Aku mencoba menanyakan sebuah pertanyaan Bapaknya tak ikut kenapa Ya aku menduga bahwa dia akan tetap membisu Tapi tidak kali ini Dia menjawab pertanyaanku dengan singkat Bapaknya selingkuh dan tak mau mempertanggungjawabkan perbuatannya jawabnya kasar Bahkan aku sempat kaget dibuatnya Bagaimana tidak Dia menjawab pertanyaanku dengan mata melotot Aku masih heran dengan tingkahnya yang terus melotot kepadaku Aku seperti dijadikan pelampiasan dendam atas peristiwa yang dia alami dan ceritakan kepadaku Tibatiba dia berkata sepatah kata singkat Tolong jaga anak ini dan kursiku Aku akan ke gerbong lima sebentar Aku tertawa manja Kupikir dia tak betah menahan air kencingnya setelah minum sebotol air mineral Tapi kenapa dia harus ke gerbong lima Ah mungkin wc di gerbong empat ini telah sesak Ya aku mengiyakan perintahnya Beberapa langkah kudengar dia meninggalkan gerbong empat Aku hanya menjawab senyumnya setelah dia memalingkan wajahnya sebentar Hawa terasa dingin saat memasuki Stasiun Pemalang Jawa Tengah Kudengar kereta sesak ini berhenti sejenak di Stasiun Pemalang Ada keanehan memang setelah petir menyambar sangat dahsyat Tapi karena penumpang semakin banyak maka kereta ini pun terus berjalan dengan kencang Aku menatap jam di tanganku saat kereta berhenti lagi di stasiun kecil di Petarukan salah satu kecamatan yang ada di Kabupaten Pemalang Hujan mulai tak bersahabat lagi bahkan suara tangisan bayi yang aku gendong kian terdengar nyaring Bayi itu seakan memanggil ibunya agar segera kembali begitulah aku mengira Sudah dua jam lebih si ibu tak kembali ucapku agak sedikit heran Terdengar beberapa suara kaki riuh dengan sepatu warna putih hitam sesekali sandal jepit murahan dengan warna hijau polos Salah satu di antara mereka ada yang beralas telapak kaki saja Aku mengira itulah sandal kulit model terbaru Tapi anehnya lagi mereka lari meninggalkan kursi saat terdengar suara letusan yang mahadahsyat Aku melihat ada tas merah di sebelahku Mungkin tas itu sengaja ditinggalkan si ibu tadi Ada secarik kertas berpita merah juga Aku hanya membaca sebagian kata tolong jaga anak kita Ternyata gerombolan orang itu berlari bukan untuk melihat macan masuk kereta atau pun melihat kembang api di tengah hujan lebat atau melihat orang gila yang menari nari riang Melainkan karena gerbong lima terlepas dari relnya dan tergulingguling di sawahnSuara benturannya mengalahkan petir Aku segera lari ke arah gerombolan orang itu sambil tetap menggendong bayi Aku menjerit kencang Raniiii Ini kali pertamanya aku menangis sampai mengeluarkan air mata penyesalan Si anak yang dari tadi kulihat bertanya Kakak nangis kenapa Kita kan selamat ataukah kakak kehilangan istri Kakak sudah kawin kan Tapi belum nikah ya Kakak ini aku bawa sapu tangan Kita menangis bersamasama setelah ini"
test_data_path = os.path.join(os.getcwd(), 'test_data')

file = '6.Percy Jackson dan Lautan Monster.txt'

Teks = open(os.path.join(test_data_path, file), encoding='cp1252')
# Teks = open(os.path.join(test_data_path, file))
Teks = Teks.read()
print(Teks)

Mimpi burukku berawal seperti ini.
Aku sedang berdiri di jalanan sunyi di sebuah kota pantai kecil. Saat itu tepat tengah malam. Badai menerjang. Dera angin dan hujan mencabik-cabik deret pepohonan palem sepanjang trotoar. Bagunan semen merah jambu dan kuning menjajari tepi jalan, jendela-jendelanya tertutupi papan. Satu blok di depan, melewati semak-semak kembang sepatu, lautan tampak bergulung ganas.

Florida, pikirku. Meski aku tak yakin bagaimana aku bisa tahu itu. Aku tak pernah menginjakkan kakiku di Florida sebelumnya.

Kemudian aku mendengar suara derap kaki hewan menapaki jalan. Aku menoleh dan melihat temanku Grover berlari terbirit-birit.

Betul, aku bilang derap kaki hewan.

Grover adalah seorang satir. Dari pinggang ke atas, dia kelihatan seperti pemuda jangkung tipikal dengan jenggot pendek berwarna oranye muda dan mengidap jerawat parah. Dia berjalan dengan pincang, tapi kecuali kau mendapatinya sedang tak memakai celana (yang sangat tak kusarankan), kau tak akan mendapa

## CASE FOLDING

In [40]:
#Case_folding = Teks.lower()
#print(Case_folding)
def Case_folding(casefold):
    casefold = casefold.lower()
    # Remove angka termasuk angka yang berada dalam string
    # Remove non ASCII chars
    casefold = re.sub(r'[^\x00-\x7f]', r'', casefold)
    casefold = re.sub(r'(\\u[0-9A-Fa-f]+)', r'', casefold)
    casefold = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", casefold)
    casefold = re.sub(r'\\u\w\w\w\w', '', casefold)
    # Remove link web
    casefold = re.sub(r'http\S+', '', casefold)
    # Remove @username
    casefold = re.sub('@[^\s]+', '', casefold)
    # Remove #tagger
    casefold = re.sub(r'#([^\s]+)', '', casefold)
    # Remove simbol, angka dan karakter aneh
    casefold = re.sub(r"[.,:;+!\-_<^/=?\"'\(\)\d\*]", " ", casefold)
    return casefold
   


Kata = Case_folding(Teks)
print (Kata)
#Case_folding = re.sub(r"\d+", "", Teks)
#print(Case_folding)

mimpi burukku berawal seperti ini  aku sedang berdiri di jalanan sunyi di sebuah kota pantai kecil  saat itu tepat tengah malam  badai menerjang  dera angin dan hujan mencabik cabik deret pepohonan palem sepanjang trotoar  bagunan semen merah jambu dan kuning menjajari tepi jalan  jendela jendelanya tertutupi papan  satu blok di depan  melewati semak semak kembang sepatu  lautan tampak bergulung ganas   florida  pikirku  meski aku tak yakin bagaimana aku bisa tahu itu  aku tak pernah menginjakkan kakiku di florida sebelumnya   kemudian aku mendengar suara derap kaki hewan menapaki jalan  aku menoleh dan melihat temanku grover berlari terbirit birit   betul  aku bilang derap kaki hewan   grover adalah seorang satir  dari pinggang ke atas  dia kelihatan seperti pemuda jangkung tipikal dengan jenggot pendek berwarna oranye muda dan mengidap jerawat parah  dia berjalan dengan pincang  tapi kecuali kau mendapatinya sedang tak memakai celana  yang sangat tak kusarankan   kau tak akan mendapa

## TOKENIZING

In [41]:
Tokenizing = Kata.split()
print(Tokenizing)
len_txt = len(Tokenizing)

['mimpi', 'burukku', 'berawal', 'seperti', 'ini', 'aku', 'sedang', 'berdiri', 'di', 'jalanan', 'sunyi', 'di', 'sebuah', 'kota', 'pantai', 'kecil', 'saat', 'itu', 'tepat', 'tengah', 'malam', 'badai', 'menerjang', 'dera', 'angin', 'dan', 'hujan', 'mencabik', 'cabik', 'deret', 'pepohonan', 'palem', 'sepanjang', 'trotoar', 'bagunan', 'semen', 'merah', 'jambu', 'dan', 'kuning', 'menjajari', 'tepi', 'jalan', 'jendela', 'jendelanya', 'tertutupi', 'papan', 'satu', 'blok', 'di', 'depan', 'melewati', 'semak', 'semak', 'kembang', 'sepatu', 'lautan', 'tampak', 'bergulung', 'ganas', 'florida', 'pikirku', 'meski', 'aku', 'tak', 'yakin', 'bagaimana', 'aku', 'bisa', 'tahu', 'itu', 'aku', 'tak', 'pernah', 'menginjakkan', 'kakiku', 'di', 'florida', 'sebelumnya', 'kemudian', 'aku', 'mendengar', 'suara', 'derap', 'kaki', 'hewan', 'menapaki', 'jalan', 'aku', 'menoleh', 'dan', 'melihat', 'temanku', 'grover', 'berlari', 'terbirit', 'birit', 'betul', 'aku', 'bilang', 'derap', 'kaki', 'hewan', 'grover', 'adala

## STOPWORD REMOVING

In [42]:
def removeStopword(remove_stop):
    
    Stopword_data = open('stopwords2.txt', 'r').read().split()
    Kata_baru = []
    content = []
    filteredtext = [word for word in remove_stop if word not in Stopword_data]
    content.append(" ".join(filteredtext))
    remove_stop = content
    return remove_stop

Kata_baru = removeStopword(Tokenizing)
print (Kata_baru)

['mimpi burukku berdiri jalanan sunyi kota pantai malam badai menerjang dera angin hujan mencabik cabik deret pepohonan palem trotoar bagunan semen merah jambu kuning menjajari tepi jalan jendela jendelanya tertutupi papan blok melewati semak semak kembang sepatu lautan bergulung ganas florida pikirku menginjakkan kakiku florida mendengar suara derap kaki hewan menapaki jalan menoleh temanku grover berlari terbirit birit bilang derap kaki hewan grover satir pinggang pemuda jangkung tipikal jenggot pendek berwarna oranye muda mengidap jerawat parah berjalan pincang kecuali kau mendapatinya memakai celana kusarankan kau kesan nonmanusia darinya celana jins longgar kaki palsu menyembunyikan fakta memiliki bokong berbulu sepasang kaki berkuku belah grover sahabatku kelas enam menyertai petualangan bersamaku gadis bernama annabeth menyelamatkan dunia bertemu grover semenjak juli misi berbahayanya dirisebuah misi ditempuh satir selamat omong omong mimpiku grover menyeret ekor kambingnya semb

## HASIL

In [43]:
hasil_stem_idris = []

Kata_baru = Kata_baru[0]

string_split = Kata_baru.split(' ')
for a in string_split:
    hasil_stem_idris.append(stemming_idris(a))
    
# hasil_stem_idris
# jumlah_kata = len_txt
jumlah_stem = len(hasil_stem_idris)
kata_benar = sum('(Kata Salah)' not in x for x in hasil_stem_idris)
kata_salah = sum('(Kata Salah)' in x for x in hasil_stem_idris)
acc = round((kata_benar/jumlah_stem)*100, 3)

print("jumlah total Kata \t= "+str(len_txt))
print("jumlah stemming Kata \t= "+str(jumlah_stem))
print("jumlah Kata benar \t= "+str(kata_benar))
print("jumlah Kata salah \t= "+str(kata_salah))
print("Akurasi stemming \t= {}%".format(acc))

jumlah total Kata 	= 2190
jumlah stemming Kata 	= 1116
jumlah Kata benar 	= 887
jumlah Kata salah 	= 229
Akurasi stemming 	= 79.48%


In [44]:
hasil_stem = [x[:-13] if 'Salah' in x else x for x in hasil_stem_idris ]
#mengambil list actual word (kata yang benar)
actual_data_path = os.path.join(os.getcwd(), 'actual_word')
actual_word = open(os.path.join(actual_data_path, file))
actual_word = [line.rstrip('\n') for line in actual_word]

In [45]:
#membuat dataframe dengan nama kolom dibawah ini
df = pd.DataFrame(columns =  ["Kata Asli", "Stemming Idris", "Stemming Asli", "Hasil"])
for i in range(len(string_split)):
    #jika kata tersebut adalah kata asing, nama kota, nama orang dsb
    if '(' in actual_word[i] and string_split[i] == hasil_stem[i] and hasil_stem[i] == actual_word[i][:actual_word[i].find('(')]:
        new_row = {'Kata Asli':string_split[i], 'Stemming Idris':hasil_stem[i], 'Stemming Asli':actual_word[i][:actual_word[i].find('(')], 'Hasil':'Bukan Kata Berimbuhan'}
        df = df.append(new_row, ignore_index=True)

    elif'(' in actual_word[i] and string_split[i] == hasil_stem[i] and hasil_stem[i] != actual_word[i][:actual_word[i].find('(')]:
        new_row = {'Kata Asli':string_split[i], 'Stemming Idris':hasil_stem[i], 'Stemming Asli':actual_word[i][:actual_word[i].find('(')], 'Hasil':'Unchange'}
        df = df.append(new_row, ignore_index=True)

    #sudah dari awal kata tersebut adalah kata dasar
    elif string_split[i] == hasil_stem[i] and hasil_stem[i] == actual_word[i]:
        new_row = {'Kata Asli':string_split[i], 'Stemming Idris':hasil_stem[i], 'Stemming Asli':actual_word[i], 'Hasil':'Kata Dasar'}
        df = df.append(new_row, ignore_index=True)

    #kata tersebut berhasil distemming dengan baik
    elif string_split[i] != hasil_stem[i] and hasil_stem[i] == actual_word[i]:
        new_row = {'Kata Asli':string_split[i], 'Stemming Idris':hasil_stem[i], 'Stemming Asli':actual_word[i], 'Hasil':'Benar'}
        df = df.append(new_row, ignore_index=True)

    #Kata tersebut tidak berubah dari awal (tidak terstemming)
    elif string_split[i] == hasil_stem[i] and hasil_stem[i] != actual_word[i]:
        new_row = {'Kata Asli':string_split[i], 'Stemming Idris':hasil_stem[i], 'Stemming Asli':actual_word[i], 'Hasil':'Unchange'}
        df = df.append(new_row, ignore_index=True)

    #kata tersebut berhasil distemming tetapi hasilnya salah
    elif string_split[i] != hasil_stem[i] and len(hasil_stem[i]) == len(actual_word[i]):
        new_row = {'Kata Asli':string_split[i], 'Stemming Idris':hasil_stem[i], 'Stemming Asli':actual_word[i], 'Hasil':'Spelling exception'}
        df = df.append(new_row, ignore_index=True)

    #kata tersebut berhasil distemming tetapi hasilnya overstemming
    elif string_split[i] != hasil_stem[i] and len(hasil_stem[i]) < len(actual_word[i]):
        new_row = {'Kata Asli':string_split[i], 'Stemming Idris':hasil_stem[i], 'Stemming Asli':actual_word[i], 'Hasil':'Overstemming'}
        df = df.append(new_row, ignore_index=True)

    #kata tersebut berhasil distemming tetapi hasilnya understemming
    elif string_split[i] != hasil_stem[i] and len(hasil_stem[i]) > len(actual_word[i]):
        new_row = {'Kata Asli':string_split[i], 'Stemming Idris':hasil_stem[i], 'Stemming Asli':actual_word[i], 'Hasil':'Understemming'}
        df = df.append(new_row, ignore_index=True)

    #jika tidak masuk semua kondisi diatas
    else:
        new_row = {'Kata Asli':string_split[i], 'Stemming Idris':hasil_stem[i], 'Stemming Asli':actual_word[i], 'Hasil':'None'}
        df = df.append(new_row, ignore_index=True)

#export ke file csv
df.to_csv('Perbandingan Stemming Idris.csv', index=False)
print("  Hasil Perbandingan Dengan Data Sebenarnya")
print("   Kata dasar benar tanpa imbuhan \t\t\t\t(benar) =", len(df[df.Hasil == 'Kata Dasar']))
print("   Kata dasar hasil stemming  \t\t\t\t\t(benar) =", len(df[df.Hasil == 'Benar']))
print("   Bukan Kata Berimbuhan \t\t\t\t\t(salah) =", len(df[df.Hasil == 'Bukan Kata Berimbuhan']))
print("   Kata yang tidak berubah(Unchange) \t\t\t\t(salah) =", len(df[df.Hasil == 'Unchange']))
print("   Kata hasil Stemming yang salah(spelling exception) \t\t(salah) =", len(df[df.Hasil == 'Spelling exception']))
print("   Kata hasil stemming yg understemming \t\t\t(salah) =", len(df[df.Hasil == 'Understemming']))
print("   Kata hasil stemming yg overstemming \t\t\t\t(salah) =", len(df[df.Hasil == 'Overstemming']), '\n')

  Hasil Perbandingan Dengan Data Sebenarnya
   Kata dasar benar tanpa imbuhan 				(benar) = 514
   Kata dasar hasil stemming  					(benar) = 336
   Bukan Kata Berimbuhan 					(salah) = 106
   Kata yang tidak berubah(Unchange) 				(salah) = 124
   Kata hasil Stemming yang salah(spelling exception) 		(salah) = 15
   Kata hasil stemming yg understemming 			(salah) = 2
   Kata hasil stemming yg overstemming 				(salah) = 19 



In [46]:
df[df.Hasil == 'Bukan Kata Berimbuhan']

,Kata Asli,Stemming Idris,Stemming Asli,Hasil
40,florida,florida,florida,Bukan Kata Berimbuhan
44,florida,florida,florida,Bukan Kata Berimbuhan
54,grover,grover,grover,Bukan Kata Berimbuhan
62,grover,grover,grover,Bukan Kata Berimbuhan
89,jins,jins,jins,Bukan Kata Berimbuhan
...,...,...,...,...
1079,sloan,sloan,sloan,Bukan Kata Berimbuhan
1081,sloan,sloan,sloan,Bukan Kata Berimbuhan
1087,jackson,jackson,jackson,Bukan Kata Berimbuhan
1105,sloan,sloan,sloan,Bukan Kata Berimbuhan


In [47]:
df[df.Hasil == 'Kata Dasar']

,Kata Asli,Stemming Idris,Stemming Asli,Hasil
0,mimpi,mimpi,mimpi,Kata Dasar
4,sunyi,sunyi,sunyi,Kata Dasar
5,kota,kota,kota,Kata Dasar
6,pantai,pantai,pantai,Kata Dasar
7,malam,malam,malam,Kata Dasar
...,...,...,...,...
1106,dengar,dengar,dengar,Kata Dasar
1107,teman,teman,teman,Kata Dasar
1108,teman,teman,teman,Kata Dasar
1110,jelek,jelek,jelek,Kata Dasar


In [48]:
df[df.Hasil == 'Benar']

,Kata Asli,Stemming Idris,Stemming Asli,Hasil
1,burukku,buruk,buruk,Benar
2,berdiri,diri,diri,Benar
3,jalanan,jalan,jalan,Benar
9,menerjang,terjang,terjang,Benar
13,mencabik,cabik,cabik,Benar
...,...,...,...,...
1098,semerah,merah,merah,Benar
1102,mencoba,coba,coba,Benar
1103,berpikir,pikir,pikir,Benar
1112,tertawa,tawa,tawa,Benar


In [49]:
df[df.Hasil == 'Unchange']

,Kata Asli,Stemming Idris,Stemming Asli,Hasil
19,bagunan,bagunan,bangun,Unchange
29,tertutupi,tertutupi,tutup,Unchange
32,melewati,melewati,lewat,Unchange
37,lautan,lautan,laut,Unchange
42,menginjakkan,menginjakkan,injak,Unchange
...,...,...,...,...
1095,tanganku,tanganku,tangan,Unchange
1096,kuharap,kuharap,harap,Unchange
1099,kurasakan,kurasakan,rasa,Unchange
1104,kukatakan,kukatakan,kata,Unchange


In [50]:
df[df.Hasil == 'Spelling exception']

,Kata Asli,Stemming Idris,Stemming Asli,Hasil
175,mengerikan,gerik,ngeri,Spelling exception
197,melarikan,arik,lari,Spelling exception
249,memuakkan,puak,muak,Spelling exception
343,seruku,ruku,seru,Spelling exception
403,melarikan,arik,lari,Spelling exception
465,merayakan,ayak,raya,Spelling exception
486,mengenakan,gena,kena,Spelling exception
515,mengerutkan,gerut,kerut,Spelling exception
521,mengeringkan,gering,kering,Spelling exception
555,merayakan,ayak,raya,Spelling exception


In [51]:
df[df.Hasil == 'Understemming']

,Kata Asli,Stemming Idris,Stemming Asli,Hasil
372,mengayunkan,gayun,ayun,Understemming
557,mengantarmu,gantar,antar,Understemming


In [52]:
df[df.Hasil == 'Overstemming']

,Kata Asli,Stemming Idris,Stemming Asli,Hasil
50,menapaki,apak,tapak,Overstemming
53,temanku,man,teman,Overstemming
313,ketukan,tuk,ketuk,Overstemming
388,memaksa,aksa,paksa,Overstemming
496,kenakan,nak,kenakan,Overstemming
514,menatapku,atap,tatap,Overstemming
564,senangi,nang,senang,Overstemming
592,memuntir,untir,puntir,Overstemming
610,menurutnya,urut,turut,Overstemming
614,menundanya,unda,tunda,Overstemming
